In [ ]:
# default_exp setup

In [77]:
import gcp_runner.core
gcp_runner.core.export_and_reload_all(silent=True)

In [76]:
#export 
from criteo_nbdev.constants import *

# Configure Google Cloud SDK
## Install Google Cloud SDK

See https://cloud.google.com/sdk/docs/downloads-interactive

In [2]:
!curl https://sdk.cloud.google.com > install.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   443  100   443    0     0   3545      0 --:--:-- --:--:-- --:--:--  3572


In [3]:
!bash install.sh --disable-prompts

######################################################################## 100.0%
Running install script from: /var/folders/0c/_9t2rb612zq01d325yh__5ph0069sb/T/tmp.XXXXXXXXXX.NqjaQrCj/install_google_cloud_sdk.bash
which curl
curl -# -f https://dl.google.com/dl/cloudsdk/channels/rapid/google-cloud-sdk.tar.gz
######################################################################## 100.0%                      4.5%     5.6%   17.6%                                30.6%                  48.7%############                  78.5%################                 80.5%

mkdir -p /Users/alekseyv
"/Users/alekseyv/google-cloud-sdk" already exists and may contain out of date files.
Remove /Users/alekseyv/google-cloud-sdk or select a new installation directory, then run again.


## Configure Project

In [ ]:
!gcloud init

In [15]:
!echo $PROJECT_ID

alekseyv-scalableai-dev


In [74]:
!gcloud config set project $PROJECT_ID

Updated property [core/project].


In [75]:
!gcloud services enable bigquery.googleapis.com
!gcloud services enable bigquerystorage.googleapis.com
!gcloud services enable container.googleapis.com
!gcloud services enable ml.googleapis.com
!gcloud services enable storage-api.googleapis.com
!gcloud services enable storage-component.googleapis.com
!gcloud services enable tpu.googleapis.com

## Download and configure service account key

See https://cloud.google.com/iam/docs/creating-managing-service-account-keys#iam-service-account-keys-create-gcloud

In [44]:
user_name = !echo $USER
iam_account = '{user_name}-service@{project_id}.iam.gserviceaccount.com'.format(user_name=user_name[0], project_id=PROJECT_ID)
iam_account

'alekseyv-service@alekseyv-scalableai-dev.iam.gserviceaccount.com'

In [50]:
!gcloud iam service-accounts keys create ./$SERVICE_ACCOUNT_KEY_FILE_NAME --iam-account=$iam_account

created key [7754ec6da57f9f5c55a471209c34b0fb640c1fa1] of type [json] as [./service_account_key.json] for [alekseyv-service@alekseyv-scalableai-dev.iam.gserviceaccount.com]


In [73]:
!gsutil cp ./$SERVICE_ACCOUNT_KEY_FILE_NAME $PRIVATE_GCS_BUCKET

Copying file://./service_account_key.json [Content-Type=application/json]...
- [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      


In [2]:
!curl https://raw.githubusercontent.com/grpc/grpc/master/etc/roots.pem -o ./roots.pem

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  277k  100  277k    0     0   110k      0  0:00:02  0:00:02 --:--:--  110k


In [3]:
!ls

00_core.ipynb            MANIFEST.in              roots.pem
01_constants.ipynb       Makefile                 service_account_key.json
02_data_download.ipynb   README.md                settings.ini
03_data_import.ipynb     criteo_nbdev             setup.ipynb
04_data_reader.ipynb     docs                     setup.py
CONTRIBUTING.md          index.ipynb
LICENSE                  install.sh


In [72]:
#export
import os
import os.path

#TODO: for cloud runs, need to copy file to cloud machine
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=os.path.join(os.getcwd(), SERVICE_ACCOUNT_KEY_FILE_NAME)
os.environ['GRPC_DEFAULT_SSL_ROOTS_FILE_PATH']=os.path.join(os.getcwd(), 'roots.pem')


## Install other dependencies

In [ ]:
!pip install google-cloud
!pip install google-cloud-bigquery
!pip install tensorflow-gpu
!pip install --no-deps tensorflow-io
!pip install matplotlib

## Configure Kubernetes (Optional)

If you plan to run sample on Google Cloud Kubernetes, you need to create a cluster, and configure it.

In Google Cloud Console for your project, navigate to Kubernetes page and create a cluster. 
Here you can select number of nodes, CPU, memory, and HDD size for each node.
You can also attach GPUs and TPUs.

Same can be achieved using ```gcloud beta container clusters create``` command, but it usually easier to do it from Console.

Once your cluster is created run following command to configure Google Cloud SDK to use it:

```sh
gcloud container clusters get-credentials <cluster name> --region=<region name>
```

If you plan to use GPUs, you also need to install NVIDIA drivers:
https://cloud.google.com/kubernetes-engine/docs/how-to/gpus#installing_drivers

```sh
kubectl apply -f https://raw.githubusercontent.com/GoogleCloudPlatform/container-engine-accelerators/master/nvidia-driver-installer/cos/daemonset-preloaded.yaml
```

If you want to use TPUs, you'll need to grant TPU permission to your service account. 

Follow this instructions:
https://cloud.google.com/tpu/docs/kubernetes-engine-setup
https://cloud.google.com/ml-engine/docs/tensorflow/using-tpus 